## Q1

Dataset:
- [`Restaurants_in_Durham_County_NC.csv`](./Restaurants_in_Durham_County_NC.csv)
- [`durham-nc-foreclosure-2006-2016.json`](./durham-nc-foreclosure-2006-2016.json)

For each restaurant (`Restaurants_in_Durham_County_NC`) with `status`=`ACTIVE` and `rpt_area_desc`=`Food Service`, show the number of foreclosures (`durham-nc-foreclosure-2006-2016`) within a radius of 1 mile of the restaurant’s coordinates.

In [1]:
import re
import json

import dask.dataframe as dd
import dask.bag as db

from haversine import haversine, Unit

In [2]:
restaurant_file_path = 'hw5/Restaurants_in_Durham_County_NC.csv'
foreclosure_file_path = 'hw5/durham-nc-foreclosure-2006-2016.json'

In [3]:
restaurant_df = dd.read_csv(restaurant_file_path, on_bad_lines='skip', delimiter=';', dtype={'Closing_Date': str, 'Hours_Of_Operation': str, 'Premise_Zip': str})

restaurant_df

,ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,Closing_Date,Seats,Water,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,object,object,object,object,float64,object,object,int64,object,int64,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
foreclosures_bag = db.read_text(foreclosure_file_path).map(json.loads).flatten()
foreclosures_df = foreclosures_bag.to_dataframe()

foreclosures_df

,datasetid,recordid,fields,geometry,record_timestamp
npartitions=1,,,,,
,object,object,object,object,object
,...,...,...,...,...


In [5]:
active_food_service_restaurants = restaurant_df.loc[(restaurant_df['Status'] == "ACTIVE") & (restaurant_df['Rpt_Area_Desc'] == "Food Service")]
active_food_service_restaurants

,ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,Closing_Date,Seats,Water,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,object,object,object,object,float64,object,object,int64,object,int64,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
get_coordinates = lambda x: x['coordinates'] if isinstance(x, dict) else None
foreclosures_df['coordinates'] = foreclosures_df['geometry'].map(get_coordinates, meta=('coordinates', 'str'))

foreclosures_df

,datasetid,recordid,fields,geometry,record_timestamp,coordinates
npartitions=1,,,,,,
,object,object,object,object,object,object
,...,...,...,...,...,...


In [7]:
merged_df = dd.merge(active_food_service_restaurants.assign(key=1), foreclosures_df.assign(key=1), on='key').drop('key', axis=1)

merged_df

,ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,Closing_Date,Seats,Water,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation,datasetid,recordid,fields,geometry,record_timestamp,coordinates
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,object,object,object,object,float64,object,object,int64,object,int64,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
def geo_lat(x):
    if isinstance(x, float):
        return 0
    return str(x).split(',')[1]

In [9]:
def geo_lon(x):
    return str(x).split(',')[0]

In [10]:
def coor_lat(x):
    if x is None:
        return 0
    return x[0]

In [11]:
def coor_lon(x):
    if x is None:
        return 0
    return x[1]

In [12]:
merged_df['lat1'] = merged_df['geolocation'].map(geo_lat, meta=('lat1', 'f8'))
merged_df['lon1'] = merged_df['geolocation'].map(geo_lon, meta=('lon1', 'f8'))
merged_df['lat2'] = merged_df['coordinates'].map(coor_lat, meta=('lat2', 'f8'))
merged_df['lon2'] = merged_df['coordinates'].map(coor_lon, meta=('lon2', 'f8'))

merged_df

,ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,Closing_Date,Seats,Water,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation,datasetid,recordid,fields,geometry,record_timestamp,coordinates,lat1,lon1,lat2,lon2
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,object,object,object,object,float64,object,object,int64,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [13]:
def haversine_dist(lat_a, lon_a, lat_b, lon_b):
    if lat_a is None or lon_a is None or lat_b is None or lon_b is None:
        return 2000.0
    return haversine((float(lat_a), float(lon_a)), (float(lat_b), float(lon_b)), unit=Unit.MILES)

In [14]:
merged_df['distance'] = merged_df.apply(lambda row: haversine_dist(row['lat1'], row['lon1'], row['lat2'], row['lon2']), axis=1, meta=('distance', 'f8'))

merged_df

,ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,Closing_Date,Seats,Water,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation,datasetid,recordid,fields,geometry,record_timestamp,coordinates,lat1,lon1,lat2,lon2,distance
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,object,object,object,object,float64,object,object,int64,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
result = merged_df.loc[merged_df['distance'] < 1.0]

In [16]:
result = result.groupby("Premise_Name").Premise_Name.count()

In [17]:
result = result.compute().sort_values(ascending=False)

In [18]:
result

Premise_Name
SUBWAY                          3332
TOP'S CHINA RESTAURANT          1057
WAFFLE HOUSE                    1041
ARBY'S                           884
KFC                              866
                                ... 
TROPICAL SMOOTHIE CAFE             3
BASF CAFETERIA                     2
FOREST VIEW LUNCHROOM              2
ROLLINGVIEW MARINA SNACK BAR       1
BIOGEN CAFE 35                     1
Name: Premise_Name, Length: 1023, dtype: int64

## Q2

Input: [`hw1dir1.zip`](./hw1dir.zip)

Solve: Conditional probability of w2 given w1, `P(w2|w1)`.

Compute the Bigram LM, of the form “word1, word2”, P(word2 | word1)

```
P(w2 | w1)
= P(w1 w2) / P(w1)
= count(w1 w2) * total(bigrams) / count(w1) * total(unigrams)
```

In [ ]:
!unzip hw1dir.zip

In [19]:
def parse_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = text.lower()
    words = text.split()
    
    if len(words)<3:
        return []
    bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]
    return bigrams

In [20]:
text_bag = db.read_text('hw1/*.txt')

In [21]:
bigrams_bag = text_bag.map(parse_text)

In [22]:
count_bag = bigrams_bag.flatten().frequencies()

In [23]:
total_bigrams = count_bag.count()

In [24]:
bigram_dict = dict(count_bag.compute())

In [25]:
bigram_total_count = total_bigrams.compute()

In [26]:
bigram_with_probs = bigram_dict

In [27]:
for key in bigram_with_probs:
    bigram_with_probs[key] = bigram_with_probs[key]/bigram_total_count

In [28]:
word1 = 'n'
word2 = 't'
prob = count_bag.filter(lambda x: x[0][0] == word1 and x[0][1] == word2).map(lambda x: x[1]/total_bigrams.compute()).sum().compute()

print(f'The conditional probability of "{word2}" given "{word1}" is {prob:.4f}')

The conditional probability of "t" given "n" is 0.0199
